In [1]:
import torch
from torch import nn
from torchvision import transforms as T
from PIL import Image
import numpy as np
from pathlib import Path
from collections import deque
import random, datetime, os, copy
%reload_ext autoreload
%autoreload 

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
import gym
from gym.spaces import Box
from gym.wrappers import FrameStack
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
import gym_super_mario_bros.actions as JoypadActions

In [4]:
from lib.env_wrappers import EnvWrapperFactory
from agents.ForgetfulAgent import ForgetfulAgent
from lib.MetricLogger import MetricLogger
%reload_ext autoreload
%autoreload 

In [5]:

imageShape = (84, 84)
actionShape = len(JoypadActions.SIMPLE_MOVEMENT)

In [6]:
%reload_ext autoreload
%autoreload 
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, JoypadActions.SIMPLE_MOVEMENT)
env = EnvWrapperFactory.convert(env, shape=imageShape)
state = env.reset()
print(state.shape)

shape before any transformations: (240, 256, 3)
shape after grayscaler: (240, 256)
shape after resizer: (84, 84)
shape after all transformations: (5, 84, 84)
(5, 84, 84)


In [7]:
env.observation_space.shape

(5, 84, 84)

In [8]:
%reload_ext autoreload
%autoreload 
# agent = ForgetfulAgent(state_shape=env.observation_space.shape, action_shape=actionShape, device=device)
agent = ForgetfulAgent(state_shape=env.observation_space.shape, action_shape=actionShape, device=device, net_name="CNN84x84")
save_dir = Path("logs") / datetime.datetime.now().strftime("%Y-%m-%dT%H-%M-%S")

save_dir.mkdir(parents=True)
logger = MetricLogger(save_dir)

In [ ]:
# try training for stage 1 only

maxStepsPerEpisode = 100_000_000
learnCount = 0
episodes = 100

for eps in range(episodes):
    state = env.reset()
    print(f"starting episode: {eps}")
    for i in range(maxStepsPerEpisode):

        action = agent.getAction(state)
        next_state, reward, done, info = env.step(action)
        
        if info["flag_get"]:
            print(f"reached a flag")
            print(info)
            reward += 1000

        # add to memory
        agent.cache(state, next_state, action, reward, done)

        q, loss = agent.learn()
        
        logger.log_step(reward, loss, q)
        
        state = next_state

        if q is not None:
            learnCount += 1
            if learnCount % 1000 == 0:
                print(f"q:{q}, loss={loss}")
        
        # if reward < 0:
        #     print(f"got negative reward({reward}) with action {action}")
        #     print(info)
            
        if done or info["flag_get"]:
            break
            
    # print(f"done: {done},\n info: {info}")
    logger.log_episode()
    if eps % 1 == 0:
        logger.record(episode=eps, epsilon=agent.exploration_rate, step=agent.current_step)
        print(info)
        

starting episode: 0
{'coins': 0, 'flag_get': False, 'life': 255, 'score': 200, 'stage': 1, 'status': 'small', 'time': 389, 'world': 1, 'x_pos': 259, 'y_pos': 79}
starting episode: 1
{'coins': 0, 'flag_get': False, 'life': 255, 'score': 200, 'stage': 1, 'status': 'small', 'time': 390, 'world': 1, 'x_pos': 291, 'y_pos': 79}
starting episode: 2
{'coins': 2, 'flag_get': False, 'life': 255, 'score': 500, 'stage': 1, 'status': 'small', 'time': 363, 'world': 1, 'x_pos': 684, 'y_pos': 85}
starting episode: 3
{'coins': 3, 'flag_get': False, 'life': 255, 'score': 1500, 'stage': 1, 'status': 'small', 'time': 378, 'world': 1, 'x_pos': 2017, 'y_pos': 79}
starting episode: 4
{'coins': 2, 'flag_get': False, 'life': 255, 'score': 500, 'stage': 1, 'status': 'small', 'time': 392, 'world': 1, 'x_pos': 297, 'y_pos': 79}
starting episode: 5
{'coins': 1, 'flag_get': False, 'life': 255, 'score': 600, 'stage': 1, 'status': 'small', 'time': 363, 'world': 1, 'x_pos': 700, 'y_pos': 79}
starting episode: 6
{'coin

In [ ]:

env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, JoypadActions.SIMPLE_MOVEMENT)
env = EnvWrapperFactory.convert(env, shape=imageShape)


done = True
count = 0 
for step in range(100000):
    if done:
        count += 1
        if count > 2:
            break
        state = env.reset()
    state, reward, done, info = env.step(agent.getAction(state))
    env.render()
env.close()

In [ ]:
def saveCheckpoint(name, epoch, model, optimizer):
    
    torch.save({
        "epoch": epoch,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "exploration_rate": agent.exploration_rate
        
    # }, f"{model.name}-checkpoint-{epoch}")
    }, f"{name}-checkpoint-{epoch}.pytorch")

In [ ]:
# save_path = (
#             save_dir / f"mario_net_{int(agent.current_step // agent.onlinePeriod)}.chkpt"
#         )
# torch.save(
#     dict(model=agent.net.state_dict(), exploration_rate=agent.exploration_rate),
#     save_path,
# )
saveCheckpoint("ForgetfulAgent-CNN50x50", 500, agent.net, agent.optimizer)

In [ ]:
agent.save(dir="", epoch=501)

In [ ]:

imageShape = (84, 84)
actionShape = len(JoypadActions.SIMPLE_MOVEMENT)

env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, JoypadActions.SIMPLE_MOVEMENT)
env = EnvWrapperFactory.convert(env, shape=imageShape)


In [ ]:
%reload_ext autoreload
%autoreload 

agent = ForgetfulAgent(state_shape=env.observation_space.shape, action_shape=actionShape, device=device, net_name="CNN84x84")
agent.load("models/ForgetfulAgent-CNN84x84-checkpoint-Apr-08-2022-9000.pytorch")


done = True
count = 0 
for step in range(1000):
    if done:
        count += 1
        if count > 2:
            break
        state = env.reset()
    state, reward, done, info = env.step(agent.getBestAction(state))
    print(f"{info['x_pos']} {info['time']}")
    if done:
        print(info)
    env.render()
env.close()

In [ ]:
env.close()